In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

import os

# %matplotlib

In [183]:
BASE_PATH = os.path.abspath(os.path.curdir)
PRICE_DATA_DIR = os.path.join(BASE_PATH, "data/price_data/")
MENTIONS_COUNT_DIR = os.path.join(BASE_PATH, "data/date_count")

COLOR_MAIN = "#363A45"
COLOR_MA_5 = "#FF9800"
COLOR_MA_8 = "#E040FB"
COLOR_MA_13 = "#9C27B0"
COLOR_VOLUME = "#B2B5BE"

MIN_DATE = "2021-02-01 00:00:00"
MAX_DATE = "2021-04-05 03:00:00"


TRACKED_COIN = "RVN"

plt.style.use("seaborn")


In [184]:
def date_parser(timestamp):
    return datetime.datetime.fromtimestamp(int(timestamp))


def add_rolling_mean(
    df, attrs: list, intervals: list, group_by=None, min_eq_max=True, min_w=1
):
    df = df.copy()
    for attr in attrs:
        for interval in intervals:
            interval *= 4
            if min_eq_max:
                min_w = interval
            if group_by != None:
                df[f"{attr}_MA_{int(interval/4)}"] = df.groupby(group_by)[
                    attr
                ].transform(lambda x: x.rolling(interval, min_w).mean())
            else:
                df[f"{attr}_MA_{int(interval/4)}"] = df[attr].transform(
                    lambda x: x.rolling(interval, min_w).mean()
                )
    return df

In [185]:
price_data = pd.read_csv(
    open(os.path.join(PRICE_DATA_DIR, TRACKED_COIN, f"{TRACKED_COIN}_price.csv"), "r"),
    parse_dates=True,
    date_parser=date_parser,
    index_col="date",
)

volume_data = pd.read_csv(
    open(os.path.join(PRICE_DATA_DIR, TRACKED_COIN, f"{TRACKED_COIN}_volume.csv"), "r"),
    parse_dates=True,
    date_parser=date_parser,
    index_col="date",
)

mentions_data = pd.read_csv(
    open(os.path.join(MENTIONS_COUNT_DIR, f"{TRACKED_COIN}.csv"), "r"),
    parse_dates=True,
    date_parser=date_parser,
    index_col="date",
)

price_data.info()
volume_data.info()
mentions_data.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1514 entries, 2021-02-01 00:00:00 to 2021-04-05 02:00:00
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   price   1514 non-null   float64
dtypes: float64(1)
memory usage: 23.7 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1514 entries, 2021-02-01 00:00:00 to 2021-04-05 02:00:00
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   volume  1514 non-null   float64
dtypes: float64(1)
memory usage: 23.7 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1516 entries, 2021-02-01 00:00:00 to 2021-04-05 03:00:00
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   count   1516 non-null   int64
dtypes: int64(1)
memory usage: 23.7 KB


In [186]:
price_data = add_rolling_mean(price_data, ["price"], [5, 8, 13])
mentions_data = add_rolling_mean(mentions_data, ["count"], [5, 8, 13])

mentions_data


,count,count_MA_5,count_MA_8,count_MA_13
date,,,,
2021-02-01 00:00:00,17,NaN,NaN,NaN
2021-02-01 01:00:00,1,NaN,NaN,NaN
2021-02-01 02:00:00,2,NaN,NaN,NaN
2021-02-01 03:00:00,1,NaN,NaN,NaN
2021-02-01 04:00:00,1,NaN,NaN,NaN
...,...,...,...,...
2021-04-04 23:00:00,0,0.05,0.03125,0.019231
2021-04-05 00:00:00,0,0.05,0.03125,0.019231
2021-04-05 01:00:00,0,0.05,0.03125,0.019231


In [188]:
fig, ax1 = plt.subplots(2, 1, figsize=(20, 14))

date_range = pd.date_range(
    start=MIN_DATE,
    end=MAX_DATE,
    freq="D",
)

ax1[0].bar(
    height=volume_data["volume"],
    x=volume_data.index,
    color=COLOR_VOLUME,
    label="Volume",
)

ax2 = ax1[0].twinx()
ax2.plot(price_data["price"], color=COLOR_MAIN, label="Price")
ax2.plot(price_data["price_MA_5"], color=COLOR_MA_5, label="Price 5h SMA")
ax2.plot(price_data["price_MA_8"], color=COLOR_MA_8, label="Price 8h SMA")
ax2.plot(price_data["price_MA_13"], color=COLOR_MA_13, label="Price 13h SMA")

ax1[1].plot(mentions_data["count_MA_5"], color=COLOR_MA_5, label="Mentios 5h SMA")
ax1[1].plot(mentions_data["count_MA_8"], color=COLOR_MA_8, label="Mentios 8h SMA")
ax1[1].plot(mentions_data["count_MA_13"], color=COLOR_MA_13, label="Mentios 13h SMA")

ax1[0].set_xlim(
    [
        datetime.datetime.fromisoformat(MIN_DATE),
        datetime.datetime.fromisoformat(MAX_DATE),
    ]
)

ax1[1].set_xlim(
    [
        datetime.datetime.fromisoformat(MIN_DATE),
        datetime.datetime.fromisoformat(MAX_DATE),
    ]
)

ax1[0].xaxis.set_ticks(date_range)
ax1[0].set_xticklabels(
    [pd.to_datetime(e).date() for e in date_range.values], rotation=45
)

ax1[1].xaxis.set_ticks(date_range)
ax1[1].set_xticklabels(
    [pd.to_datetime(e).date() for e in date_range.values], rotation=45
)

h1, l1 = ax1[0].get_legend_handles_labels()
h2, l2 = ax2.get_legend_handles_labels()
ax1[0].legend(h1 + h2, l1 + l2, loc=2)

ax1[1].legend(loc=0)

fig.suptitle(TRACKED_COIN)


Text(0.5, 0.98, 'RVN')